In [1]:
import io
from PIL import Image
import pandas as pd
import requests

import config

In [2]:
sheet_url = config.IMAGES_URL
response = requests.get(sheet_url)
content = response.content.decode()
df = pd.read_csv(io.StringIO(content))
df.dropna(subset=['image_url'], inplace=True)

In [3]:
import aiohttp
import asyncio

async def get_image_size(session, url):
    async with session.get(url) as a_response:
        if 'image' in a_response.headers.get('content-type', ''):
            image_data = await a_response.read()
            image = Image.open(io.BytesIO(image_data))
            image_size = image.size
            return f"{image_size[0]}X{image_size[1]}"
        else:
            return None

async def main(df):
    async with aiohttp.ClientSession() as session:
        tasks = [get_image_size(session, url) for url in df["image_url"]]
        image_sizes = await asyncio.gather(*tasks)
        df["SIZE"] = image_sizes
    return df

df = await main(df)

In [4]:
df

,image_url,SIZE
0,https://data.sanitino.eu/PRODUCT-33916/8607663...,1080X1614
1,https://data.sanitino.eu/PRODUCT-62434/f7aa3c2...,1080X1080
2,https://data.sanitino.eu/PRODUCT-33915/8607663...,1080X1614
3,https://data.sanitino.eu/PRODUCT-62426/506d67b...,1080X1080
4,https://data.sanitino.eu/PRODUCT-33893/ccd99b4...,1080X1614
...,...,...
46883,https://data.sanitino.eu/PRODUCT-113844/9786df...,2301X1080
46884,https://data.sanitino.eu/PRODUCT-20240/4a457da...,1621X1080
46885,https://data.sanitino.eu/PRODUCT-47765/58a1a3d...,1080X1218
46886,https://data.sanitino.eu/PRODUCT-56372/764c369...,1080X1080


In [6]:
df["SIZE"].isnull().sum()

339

In [9]:
df.dropna(subset=['SIZE'], inplace=True)

In [10]:
df["SIZE"].isnull().sum()

0

In [64]:
df

,image_url,SIZE
0,https://data.sanitino.eu/PRODUCT-33916/8607663...,1080X1614
1,https://data.sanitino.eu/PRODUCT-62434/f7aa3c2...,1080X1080
2,https://data.sanitino.eu/PRODUCT-33915/8607663...,1080X1614
3,https://data.sanitino.eu/PRODUCT-62426/506d67b...,1080X1080
4,https://data.sanitino.eu/PRODUCT-33893/ccd99b4...,1080X1614
...,...,...
46883,https://data.sanitino.eu/PRODUCT-113844/9786df...,2301X1080
46884,https://data.sanitino.eu/PRODUCT-20240/4a457da...,1621X1080
46885,https://data.sanitino.eu/PRODUCT-47765/58a1a3d...,1080X1218
46886,https://data.sanitino.eu/PRODUCT-56372/764c369...,1080X1080


In [ ]:
from writer import write_df_to_sheet

write_df_to_sheet("task_1", df)